#### Notes
- unzip address points gdb before running script
- process of elimination is used to wittle out parcels
- owned unit groupings (OUG) relies parcel common areas
    - if those aren't are accurately attributed, parcels within won't be correctly attributed
    
#### uncategorized types
- 106
- 510 Comm Imps in Res Zone
- 911 vacant lots
- 957 Related Parcel
- 997 Residential - Multi

In [1]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

In [2]:
# tag_expression and code block for tagging parcels
tag_expression = "createTag(!TAG!, '{}')"
tag_codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

In [3]:
# input taz shapefile and parcels
taz_shp = '.\\Inputs\\TAZ.shp'
parcels = '.\\Inputs\\Utah_Salt_Lake_County_Parcels_LIR.gdb\\Parcels_SaltLake_LIR_UTM12'


In [4]:
# create output gdb
outputs = '.\\Outputs'
gdb = os.path.join(outputs, "saltlake.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "saltlake.gdb")
    
scratch = os.path.join(outputs, "scratch.gdb")
if not arcpy.Exists(scratch):
    arcpy.CreateFileGDB_management(outputs, "scratch.gdb")

In [5]:
# get address points without base address point
address_pts_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\AddressPoints.gdb\\AddressPoints','address_pts_lyr')
query = (''' PtType <> 'BASE ADDRESS' ''')
arcpy.SelectLayerByAttribute_management(address_pts_lyr, 'NEW_SELECTION', query)
address_pts_no_base = arcpy.FeatureClassToFeatureClass_conversion(address_pts_lyr, gdb, '_00_address_pts_no_base')

In [6]:
# select parcels within modeling area
parcels_layer = arcpy.MakeFeatureLayer_management(parcels, 'parcels') 
arcpy.SelectLayerByLocation_management(parcels_layer, 'HAVE_THEIR_CENTER_IN', taz_shp)
parcels_for_modeling = arcpy.FeatureClassToFeatureClass_conversion(parcels_layer, gdb, '_01_parcels_for_modeling')


# add a tag field for parcel type
arcpy.AddField_management(parcels_for_modeling, 'TAG', 'TEXT')
arcpy.AddField_management(parcels_for_modeling, 'TYPE_WFRC', 'TEXT')
arcpy.AddField_management(parcels_for_modeling, 'SUBTYPE_WFRC', 'TEXT')

# count parts and rings
arcpy.AddField_management(parcels_for_modeling, 'PARTS', 'LONG')
arcpy.AddField_management(parcels_for_modeling, 'RINGS', 'LONG')

fields = ["shape@", 'PARTS', 'RINGS']
with arcpy.da.UpdateCursor(parcels_for_modeling, fields) as cursor:
    for row in cursor:
        shape = row[0]
        parts = shape.partCount
        rings = shape.boundary().partCount   
        row[1] = parts
        row[2] = rings
        cursor.updateRow(row)


# create the master layer for selection
parcels_for_modeling_layer = arcpy.MakeFeatureLayer_management(parcels_for_modeling, 'parcels_for_modeling_layer')

# get a count of all parcels
count_all = arcpy.GetCount_management(parcels_for_modeling_layer)
print("# initial parcels in modeling area:\n {}".format(count_all))

# # create centroids for all parcels
# parcels_for_modeling_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
#                                                                  os.path.join(gdb, '_01b_parcels_for_modeling_centroids'), 
#                                                                  "INSIDE")     

# initial parcels in modeling area:
 384322


In [7]:
###########
# churches
###########

query = (""" PROP_CLASS = 'ER' OR PROP_TYPE =  '956' """)
tag="churches"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

928 "churches" parcels were selected.
383394 parcels remain...


In [8]:
###############
# agriculture
###############

query = """ PROP_TYPE IN ('811','812','816','817', '818','830','850') """
tag="agriculture"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1299 "agriculture" parcels were selected.
382095 parcels remain...


In [ ]:
###############
# open space
###############

# get parcels on parks
parks = ".\\Inputs\\ParksLocal.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=parks,
                                       selection_type='NEW_SELECTION')

# get parcels on public lands that are not military
land_ownership_lyr = arcpy.MakeFeatureLayer_management('./Inputs/UT_SITLA_Ownership_LandOwnership_WM.shp','land_ownership_lyr') 

query= (""" OWNER <> 'Private' And DESIG <> 'Military' """)
arcpy.SelectLayerByAttribute_management(land_ownership_lyr, 'NEW_SELECTION', query)

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=land_ownership_lyr,
                                       selection_type='ADD_TO_SELECTION')

# get cemetery parcels using overlay
cemeteries_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\Cemeteries.shp','cemeteries_lyr') 
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=cemeteries_lyr,
                                       selection_type='ADD_TO_SELECTION')

# get parcels that are golf courses using overlay
golf_courses = ".\\Inputs\\GolfCourses.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=golf_courses,
                                       selection_type='ADD_TO_SELECTION')

# query parcel attributes
query= (""" PROP_TYPE IN ('951','960','961') OR TAXEXEMPT_TYPE = 'CM' """)
tag = "open_space"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
###############
# health care
###############

query = (""" PROP_TYPE in ('524','547','560','561') """)
tag = 'healthcare'

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
###############
# education
###############

query = (""" PROP_TYPE =  '954' OR PROP_TYPE =  '577' OR TAXEXEMPT_TYPE = 'EE' """)
tag="education"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
###############
# government
###############

query = (""" PROP_TYPE IN ('570','953','955') """)
tag="government"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
###############
# retail
###############
query=(""" PROP_TYPE IN ('513','514','515','516','528','529','530','536','548','549','551','553','562','564','573','574',
            '575','581','582','583','584','585','591','675','749','775','914') """)
tag="retail"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
###############
# industrial
###############

query = (""" PROP_TYPE IN ('200','202','203', '550', '556','695','795','915') OR PROP_CLASS = 'I' """)
tag="industrial"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
###############
# office
###############

query= ("""PROP_TYPE IN ('566','590','760','916') """)
tag="office"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
################
# single_family
################

query = (''' PROP_TYPE IN ('103','104','111', '118', '888','994','998') ''')
tag="single_family"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))


#should count address points in resulting features and look for duplexes/mh

In [ ]:
########################
# single_family w/ ADU
########################

query = (''' PROP_TYPE IN ('106') ''')
tag="single_family_adu"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'single_family_adu'",
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))


#should count address points in resulting features and look for duplexes/mh

In [ ]:
################
# duplexes
################

query = (''' PROP_TYPE IN ('112') ''')
tag="duplex"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
###############
# utilities
###############

# select parcels using query
query = (''' PROP_TYPE IN ('950') ''')
tag="utilities"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
#################
# group quarters
#################

query= (""" PROP_TYPE IN ('105','540','576') """)
tag = "group_quarters"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
######################
# Apartments (yikes!)
######################


query= (""" PROP_TYPE IN ('110','113','114', '115', '120', '150', '199') """)
tag = "apartments"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="!PROP_TYPE!",
                                expression_type="PYTHON3")

# create the feature class for the parcel type
apt_commons = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_03_{}'.format(tag))


##############

query = """ PROP_TYPE IN ('713') """

# filter for common areas
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")


common_areas =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_03a_apt_commons')

# fill in holes via eliminate polygon part
apt_commons2 = arcpy.EliminatePolygonPart_management(common_areas, os.path.join(scratch, '_03b_apt_common_areas_filled'),
                                                      condition='PERCENT', part_area_percent=60, part_option='CONTAINED_ONLY')


#=========================
# get merge condo commons
#=========================

merged_apt_commons = arcpy.Merge_management([apt_commons, apt_commons2], 
                                              os.path.join(scratch, '_03c_merged_apt_commons'))

# select all features within merged dataset and delete

#==================================================
# summarize units attributes within common areas
#==================================================

# get parcels that are contained by apt common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=merged_apt_commons, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
apt_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03d_apt_centroids'), "INSIDE")




# use spatial join to summarize market value & acreage
target_features = merged_apt_commons
join_features = apt_centroids
output_features = os.path.join(scratch, "_03e_apt_oug_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# parcel acres
fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "CONTAINS")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!".format(tag),
                                expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_apartment")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")



#=========================
# final summary
#=========================

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=merged_apt_commons, selection_type='NEW_SELECTION')
# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
################################
# OUG - Condos (double yikes!)
################################

tag = "condo"

#=====================
# get condo commons 1
#=====================

query = """ PROP_TYPE IN ('711') """

# filter for common areas, some small office parks and a few other strange cases may be present
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

common_areas =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_04a_condo_common_areas')

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# fill in holes via eliminate polygon part
common_areas_filled = arcpy.EliminatePolygonPart_management(common_areas, os.path.join(scratch, '_04b_condo_common_areas_filled'),
                                                      condition='PERCENT', part_area_percent=80, part_option='CONTAINED_ONLY')

#=====================
# get condo commons 2
#=====================
query = """ PROP_TYPE IN ('116') """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
condos =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_04c_condo_units')

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=condos,
                                       selection_type='NEW_SELECTION')
query = """ PROP_TYPE IN ('116') """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'REMOVE_FROM_SELECTION', query)
common_areas2 =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_04d_condo_common_areas2')

common_areas2_filled = arcpy.EliminatePolygonPart_management(common_areas2, os.path.join(scratch, 
                                                                                         '_04e_condo_common_areas2_filled'),
                                                      condition='PERCENT', part_area_percent=80, part_option='CONTAINED_ONLY')

common_areas2_filled_lyr = arcpy.MakeFeatureLayer_management(common_areas2_filled, 'common_areas2_filled_lyr')
arcpy.SelectLayerByLocation_management(in_layer=common_areas2_filled_lyr,overlap_type="COMPLETELY_CONTAINS",
                                       select_features=condos,
                                       selection_type='NEW_SELECTION')
common_areas3 = arcpy.FeatureClassToFeatureClass_conversion(common_areas2_filled_lyr, scratch, '_04f_condo_common_areas3')


#=====================
# get loose condos 
#=====================

query = """ PROP_TYPE IN ('116') """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

#modify selection to parcels that aren't part of the earlier subsets common_areas_filled and common_Areas3
arcpy.SelectLayerByLocation_management(in_layer=common_areas2_filled_lyr,overlap_type="WITHIN",
                                       select_features=condos,
                                       selection_type='REMOVE_FROM_SELECTION')

loose_condos = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_04g_loose_condos')

#=========================
# get merge condo commons
#=========================

merged_condo_commons = arcpy.Merge_management([common_areas_filled, common_areas3, loose_condos], 
                                              os.path.join(scratch, '_04h_merged_common_areas'))

merged_condo_commons_dissolved = arcpy.Dissolve_management(merged_condo_commons, 
                                                   os.path.join(scratch, '_04i_merged_condo_commons_dissolved'), 
                                                           multi_part='SINGLE_PART')

#==================================================
# summarize units attributes within common areas
#==================================================

# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=merged_condo_commons_dissolved, selection_type='NEW_SELECTION')

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# convert condo parcels that are contained by common areas into centroids
condo_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_04j_condos_centroids'), "INSIDE")


# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=merged_condo_commons, selection_type='NEW_SELECTION')
count_type = int(count_type[0]) + int(arcpy.GetCount_management(parcels_for_modeling_layer)[0])
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)



# use spatial join to summarize market value & acreage
target_features = merged_condo_commons_dissolved
join_features = condo_centroids
output_features = os.path.join(scratch, "_04k_condo_oug_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# parcel acres
fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "CONTAINS")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!".format(tag),
                                expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj, "Join_Count")

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)



#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_condo")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")



# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
#####################
# low income housing
#####################

query= (""" PROP_TYPE IN ('142') """)
tag = "low_income_housing"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
features = arcpy.EliminatePolygonPart_management(parcels_for_modeling_layer, os.path.join(scratch, '_06a_{}'.format(tag)),
                                                      condition='PERCENT', part_area_percent=60, part_option='CONTAINED_ONLY')


# roundabout way to eliminate individual units inside low income common areas
layer = arcpy.MakeFeatureLayer_management(features, 'layer') 
query= (""" TAXEXEMPT_TYPE = 'MC' """)
arcpy.SelectLayerByAttribute_management(layer, 'NEW_SELECTION', query)
common_areas = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_06b_{}_commons'.format(tag))
arcpy.SelectLayerByLocation_management(in_layer=layer,overlap_type="COMPLETELY_WITHIN",
                                       select_features=common_areas,
                                       selection_type='NEW_SELECTION')
layer = arcpy.DeleteFeatures_management(layer)

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = features
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_LowIncome")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
#################
# OUG - mobile home parks
#################

tag="mobile_home"

# use overlay to select mobile home parks parcels
mobile_home_parks = ".\\Inputs\\Mobile_Home_Parks.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=mobile_home_parks,
                                       selection_type='NEW_SELECTION')

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
mhp = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_07a_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = mhp
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_mobile_home_park")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")


# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))


In [ ]:
######################
# OUG - PUDs
######################

tag = "pud"

#=====================
# get condo commons 2
#=====================
query = """ PROP_TYPE IN ('119', '922') """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
puds =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_05a_pud_units')

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=puds,
                                       selection_type='NEW_SELECTION')

query = """ PROP_TYPE IN ('700', '701', '119', '922') """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'SUBSET_SELECTION', query)
common_areas =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_05b_pud_commons')
common_areas_dissolved = arcpy.Dissolve_management(parcels_for_modeling_layer, 
                                                   os.path.join(scratch, '_05c_pud_commons_dissolve'), multi_part='SINGLE_PART')


#==================================================
# summarize units attributes within pud areas
#==================================================

# parcels that are contained by pud common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=common_areas_dissolved, selection_type='NEW_SELECTION')

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# convert pud parcels that are contained by common areas into centroids
pud_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_05c_pud_centroids'), "INSIDE")


# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=common_areas_dissolved, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=common_areas, selection_type='ADD_TO_SELECTION')
count_type = int(count_type[0]) + int(arcpy.GetCount_management(parcels_for_modeling_layer)[0])
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)



# use spatial join to summarize market value & acreage
target_features = common_areas_dissolved
join_features = pud_centroids
output_features = os.path.join(scratch, "_05d_pud_oug_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# parcel acres
fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "CONTAINS")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")


# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!".format(tag),
                                expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj, "Join_Count")


# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_pud")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
#################
# commercial
#################

query=""" PROP_TYPE IN ('500','592', '593', '594', '595', '596') or PROP_CLASS = 'C' """
tag="commercial"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
#################
# roads
#################

query=""" BLDG_SQFT = 0 And FLOORS_CNT = 0 And TOTAL_MKT_VALUE = 0 And PARCEL_ADD IS NULL """
tag="road"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
#################
# Vacant land
#################

query= (""" PROP_TYPE IN ('901','902','903', '905', '906', '908', '911', '917') """)
tag = "vacant"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
# export remaining parcels
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_09_unclassed_parcels')